## Combining 2 prompts dependent on each other

In [13]:
import os
from langchain import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

In [14]:
os.environ["OPENAI_API_KEY"] = ""

In [15]:
# Using Azure Open AI key

OPENAI_API_VERSION = "2023-05-15"
AZURE_OPENAI_ENDPOINT = ""
AZURE_OPENAI_API_KEY = ""

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY

model_response_generation = 'Test'
llm = AzureChatOpenAI(openai_api_version="2023-05-15", azure_deployment=model_response_generation, )

In [46]:
capital_template = PromptTemplate(input_variables=['country'], template="What is the capital of {country}")
famous_template = PromptTemplate(input_variables=['capital'], template="Suggest me some famous places to visit in {capital}")
print(capital_template.format(country = 'India'))
print(famous_template.format(capital = 'Delhi'))

What is the capital of India
Suggest me some famous places to visit in Delhi


In [47]:
capital_chain = LLMChain(llm=llm, prompt=capital_template)
famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [45]:
chain = SimpleSequentialChain(chains = [capital_chain, famous_chain])
chain.run("India")

"1. India Gate: This is an iconic monument dedicated to the Indian soldiers who lost their lives in World War I.\n\n2. Red Fort: This is a historic fort in the city of Delhi, which served as the main residence of the Mughal emperors.\n\n3. Qutub Minar: This is the tallest brick minaret in the world, and an important example of Indo-Islamic Architecture.\n\n4. Lotus Temple: Known for its flowerlike shape, it is a prominent attraction in the city. It is a Bahá'í House of Worship.\n\n5. Humayun's Tomb: This is the tomb of the Mughal Emperor Humayun. It was the first garden-tomb on the Indian subcontinent.\n\n6. Akshardham Temple: This is a Hindu temple, and a spiritual-cultural campus in Delhi. \n\n7. Jama Masjid: This is one of the largest mosques in India.\n\n8. Rashtrapati Bhawan: This is the official home of the president of India.\n\n9. Raj Ghat: This is a memorial dedicated to Mahatma Gandhi.\n\n10. National Museum: This is one of the largest museums in India.\n\n11. The National Ga

# Sequential Chain

In [4]:
from langchain.chains import SequentialChain

In [5]:
capital_template = PromptTemplate(input_variables=['country'], template="What is the capital of {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")

famous_template = famous_template = PromptTemplate(input_variables=['capital'], template="Suggest me some famous places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="places")

In [6]:
chain = SequentialChain(chains=[capital_chain, famous_chain], input_variables=["country"], 
                        output_variables=["capital", "places"])
chain({"country": "India"})

/media/anang/New Volume/home/virtual_envs/llm_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'India',
 'capital': 'The capital of India is New Delhi.',
 'places': "1. India Gate: It's a war memorial located astride the Rajpath, on the eastern edge of the ceremonial axis of New Delhi, formerly called Kingsway.\n\n2. Humayun's Tomb: This is the tomb of the Mughal Emperor Humayun. The tomb was commissioned by Humayun's first wife and chief consort, Empress Bega Begum, in 1569-70, and designed by Mirak Mirza Ghiyas and his son, Sayyid Muhammad.\n\n3. Lotus Temple: Notable for its flowerlike shape, it has become a prominent attraction in the city. The Lotus Temple has won numerous architectural awards and been featured in hundreds of newspaper and magazine articles.\n\n4. Qutub Minar: A UNESCO World Heritage Site, this 73-meter tall tower was built by Qutub-ud-Din Aibak in 1193. \n\n5. Akshardham Temple: Also known as the Swaminarayan Akshardham, this spiritual-cultural campus in New Delhi is an epitome of Indian culture, spirituality, and architecture. \n\n6. Red Fort:

# Chat Models with ChatOpenAI

In [3]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [4]:
os.environ["OPENAI_API_KEY"] = ""

In [5]:
chatllm = ChatOpenAI(open_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6, model="gpt-3.5-turbo")

/media/anang/New Volume/home/virtual_envs/llm_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! open_api_key is not default parameter.
                    open_api_key was transferred to model_kwargs.
                    Please confirm that open_api_key is what you intended.


In [ ]:
chatllm([
    SystemMessage(content= "You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

# Prompt template + LLM + Output Parsers

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [9]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [10]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)])

In [16]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [17]:
chain.invoke({"text":"intelligent"})

['Smart', ' Brilliant', ' Clever', ' Knowledgeable', ' Wise']

# Using multiple inputs in prompt template

In [33]:
## Language Translation
from langchain import PromptTemplate

template='''In an easy way translate the following sentence '{sentence}' into {target_language}'''
language_prompt = PromptTemplate(
    input_variables=["sentence",'target_language'],
    template=template,
)
language_prompt.format(sentence="How are you",target_language='hindi')

"In an easy way translate the following sentence 'How are you' into hindi"

In [34]:
chain3=LLMChain(llm=llm,prompt=language_prompt)
chain3({'sentence': "Hello How are you", 'target_language':'hindi'})

{'sentence': 'Hello How are you',
 'target_language': 'hindi',
 'text': 'नमस्ते, आप कैसे हैं?'}

# Using Few shot template technique

In [15]:
from langchain import PromptTemplate, FewShotPromptTemplate

# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [16]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input\n",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym: ",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [17]:
print(few_shot_prompt.format(input='big'))

Give the antonym of every input

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: big
Antonym: 


In [19]:
chain=LLMChain(llm=llm,prompt=few_shot_prompt)
chain({'input':"big"})

{'input': 'big', 'text': ' small'}